In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [3]:
sc = SparkContext()
spark = SQLContext(sc)

# 여러 파일 합쳐서 read

- 파일이 연도별로 세분화 되어 있음 -> DataFrame으로 한 번에 불러오자.
- csv 수정 : header 제외한 row 제거, (skiprows 실패)

In [4]:
basic_folder = '/home/user/dataset/'

In [5]:
provinces = ['강원도','경기도','경상남도','경상북도','광주광역시','대구광역시','대전광역시','부산광역시','서울특별시','세종특별자치시','울산광역시','인천광역시','전라남도','전라북도','제주특별자치도','충청남도','충청북도']

In [6]:
for p in provinces:
    files = basic_folder + p + '/' + p + '_*' # '*'표시로 다중 파일 읽기
    print(files)
    df = spark.read.csv(files, header=True, inferSchema=True)
    print(df.count())
    df.createOrReplaceTempView('`' + p + '`')

/home/user/dataset/강원도/강원도_*
216039
/home/user/dataset/경기도/경기도_*
1791446
/home/user/dataset/경상남도/경상남도_*
462646
/home/user/dataset/경상북도/경상북도_*
315417
/home/user/dataset/광주광역시/광주광역시_*
275859
/home/user/dataset/대구광역시/대구광역시_*
386890
/home/user/dataset/대전광역시/대전광역시_*
248572
/home/user/dataset/부산광역시/부산광역시_*
537189
/home/user/dataset/서울특별시/서울특별시_*
885855
/home/user/dataset/세종특별자치시/세종특별자치시_*
37604
/home/user/dataset/울산광역시/울산광역시_*
198926
/home/user/dataset/인천광역시/인천광역시_*
408724
/home/user/dataset/전라남도/전라남도_*
200231
/home/user/dataset/전라북도/전라북도_*
241104
/home/user/dataset/제주특별자치도/제주특별자치도_*
38069
/home/user/dataset/충청남도/충청남도_*
309887
/home/user/dataset/충청북도/충청북도_*
223512


In [43]:
df.describe()

DataFrame[summary: string, 시군구: string, 단지명: string, 전용면적(㎡): string, 계약년월: string, 거래금액(만원): string, 층: string, 건축년도: string, 도로명: string]

In [8]:
df.head(5)

[Row(시군구='충청북도 괴산군 괴산읍 대사리', 단지명='두진백로', 전용면적(㎡)=39.8, 계약년월=202003, 거래금액(만원)='4,500', 층=5, 건축년도=1998, 도로명='읍내로 111-1'),
 Row(시군구='충청북도 괴산군 괴산읍 대사리', 단지명='두진백로', 전용면적(㎡)=39.8, 계약년월=202004, 거래금액(만원)='4,500', 층=4, 건축년도=1998, 도로명='읍내로 111-1'),
 Row(시군구='충청북도 괴산군 괴산읍 대사리', 단지명='두진백로', 전용면적(㎡)=39.8, 계약년월=202007, 거래금액(만원)='5,200', 층=6, 건축년도=1998, 도로명='읍내로 111-1'),
 Row(시군구='충청북도 괴산군 괴산읍 대사리', 단지명='두진백로', 전용면적(㎡)=39.8, 계약년월=202007, 거래금액(만원)='4,000', 층=6, 건축년도=1998, 도로명='읍내로 111-1'),
 Row(시군구='충청북도 괴산군 괴산읍 대사리', 단지명='두진백로', 전용면적(㎡)=39.8, 계약년월=202010, 거래금액(만원)='4,300', 층=4, 건축년도=1998, 도로명='읍내로 111-1')]

# 연도별 평균 가격

In [72]:
yearsDF = [] # [province, ...]
for province in provinces:
    query = f"""
            select substr(`계약년월`, 1, 4) as y, 
                   cast(avg(replace(`거래금액(만원)`, ',', '')) as decimal) as price
            from `{province}`
            group by y
            order by y 
            """
    temp = spark.sql(query)
    yearsDF.append((province, temp))

for province, yearDF in yearsDF:
    print(f'===== {province} =====')
    yearDF.show()

==강원도==
+----+-----+
|   y|price|
+----+-----+
|2010|10692|
|2011|11217|
|2012|10394|
|2013|10603|
|2014|10955|
|2015|12381|
|2016|13574|
|2017|14020|
|2018|14081|
|2019|14182|
|2020|15618|
|2021|16538|
+----+-----+

==경기도==
+----+-----+
|   y|price|
+----+-----+
|2010|23815|
|2011|24412|
|2012|23964|
|2013|25488|
|2014|26269|
|2015|27818|
|2016|29672|
|2017|32527|
|2018|34967|
|2019|36777|
|2020|39078|
|2021|39648|
+----+-----+

==경상남도==
+----+-----+
|   y|price|
+----+-----+
|2010|14244|
|2011|16996|
|2012|15932|
|2013|16872|
|2014|17851|
|2015|18161|
|2016|17888|
|2017|17647|
|2018|17759|
|2019|19379|
|2020|22046|
|2021|19448|
+----+-----+

==경상북도==
+----+-----+
|   y|price|
+----+-----+
|2010|10260|
|2011|12202|
|2012|12170|
|2013|12729|
|2014|13499|
|2015|13979|
|2016|13402|
|2017|14153|
|2018|13972|
|2019|14737|
|2020|16943|
|2021|15464|
+----+-----+

==광주광역시==
+----+-----+
|   y|price|
+----+-----+
|2010|12565|
|2011|13962|
|2012|14532|
|2013|15158|
|2014|16360|
|2015|17280|
|20

# 평수 별 평균 가격 query

In [83]:
sizesDF = [] # [[province [small], [mid], [big]], [], [], ...]
sizes = {'small' : (0, 70), 'middle' : (71 ,120), 'big' : (120, 10000)}
for province in provinces:
    province_list = []
    for size in sizes:
        query = f"""
                select substr(`계약년월`, 1, 4) as y, 
                       cast(avg(replace(`거래금액(만원)`, ',', '')) as decimal) as price
                from `{province}`
                where cast(`전용면적(㎡)` as decimal) between {sizes[size][0]} and {sizes[size][1]}
                group by y
                order by y 
                """
        temp = spark.sql(query)
        province_list.append((size, temp))
    sizesDF.append((province, province_list))

for province, sizeDF in sizesDF:
    print(f'===== {province} =====')
    for size, each_DF in sizeDF:
        print(f' = {size} =')
        each_DF.show()

== 강원도 ==
 = small =
+----+-----+
|   y|price|
+----+-----+
|2010| 5778|
|2011| 6590|
|2012| 6974|
|2013| 7250|
|2014| 7592|
|2015| 8191|
|2016| 8999|
|2017| 9539|
|2018| 9508|
|2019| 9282|
|2020|10058|
|2021|10834|
+----+-----+

 = middle =
+----+-----+
|   y|price|
+----+-----+
|2010|14114|
|2011|14961|
|2012|15012|
|2013|15592|
|2014|15734|
|2015|17045|
|2016|18792|
|2017|19452|
|2018|19254|
|2019|19191|
|2020|21755|
|2021|23146|
+----+-----+

 = big =
+----+-----+
|   y|price|
+----+-----+
|2010|24428|
|2011|24838|
|2012|24886|
|2013|23216|
|2014|23821|
|2015|24590|
|2016|26336|
|2017|28410|
|2018|28275|
|2019|26617|
|2020|28733|
|2021|29334|
+----+-----+

== 경기도 ==
 = small =
+----+-----+
|   y|price|
+----+-----+
|2010|15999|
|2011|16781|
|2012|16296|
|2013|17535|
|2014|18292|
|2015|19653|
|2016|21236|
|2017|23149|
|2018|24627|
|2019|25611|
|2020|28071|
|2021|27782|
+----+-----+

 = middle =
+----+-----+
|   y|price|
+----+-----+
|2010|27534|
|2011|27971|
|2012|27110|
|2013|28596

+----+-----+
|   y|price|
+----+-----+
|2010|11903|
|2011|14177|
|2012|13628|
|2013|14337|
|2014|15017|
|2015|15511|
|2016|16663|
|2017|17937|
|2018|19111|
|2019|20313|
|2020|22379|
|2021|21758|
+----+-----+

 = big =
+----+-----+
|   y|price|
+----+-----+
|2010|25019|
|2011|28383|
|2012|26876|
|2013|26593|
|2014|26952|
|2015|26130|
|2016|27556|
|2017|30452|
|2018|31861|
|2019|35113|
|2020|37006|
|2021|34152|
+----+-----+

== 전라북도 ==
 = small =
+----+-----+
|   y|price|
+----+-----+
|2010| 6578|
|2011| 7617|
|2012| 8155|
|2013| 8705|
|2014| 8623|
|2015| 8724|
|2016| 8892|
|2017| 9231|
|2018| 9974|
|2019| 9296|
|2020| 9832|
|2021| 9692|
+----+-----+

 = middle =
+----+-----+
|   y|price|
+----+-----+
|2010|13897|
|2011|15582|
|2012|15515|
|2013|16747|
|2014|17303|
|2015|18010|
|2016|18655|
|2017|19186|
|2018|19634|
|2019|20112|
|2020|23246|
|2021|20986|
+----+-----+

 = big =
+----+-----+
|   y|price|
+----+-----+
|2010|26926|
|2011|29531|
|2012|30010|
|2013|28937|
|2014|29444|
|2015|29

# 시/군/구 별 평균 가격

In [107]:

citiesDF = []
for province in provinces:
    province_list = []
    query = f"""
            select distinct substr(`시군구`, 1, locate(' ', `시군구`, instr(`시군구`, ' ') + 1) - 1) as r 
            from `{province}`
            """
    cities = spark.sql(query).rdd.flatMap(lambda x : x).collect()
    for city in cities:
        query = f"""
                select substr(`계약년월`, 1, 4) as y, 
                       cast(avg(replace(`거래금액(만원)`, ',', '')) as decimal) as price
                from `{province}`
                where `시군구` like '{city}%'
                group by y
                order by y 
                """
        temp = spark.sql(query)
        province_list.append((city, temp))
    citiesDF.append((province, province_list))

for province, cityDF in citiesDF:
    print(f'===== {province} =====')
    for city, each_DF in cityDF:
        print(f' = {city} =')
        each_DF.show()

   == 강원도 ==
 = 강원도 태백시 =
+----+-----+
|   y|price|
+----+-----+
|2010|10109|
|2011| 9755|
|2012| 8201|
|2013| 5377|
|2014| 6439|
|2015| 8214|
|2016| 9667|
|2017| 9871|
|2018| 9849|
|2019| 9188|
|2020| 8612|
|2021| 8613|
+----+-----+

 = 강원도 원주시 =
+----+-----+
|   y|price|
+----+-----+
|2010|12747|
|2011|12562|
|2012|11110|
|2013|11775|
|2014|11785|
|2015|13393|
|2016|14642|
|2017|15345|
|2018|15554|
|2019|15228|
|2020|16409|
|2021|17333|
+----+-----+

 = 강원도 철원군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 7349|
|2011| 7376|
|2012| 8585|
|2013| 8031|
|2014| 8451|
|2015| 6891|
|2016|11948|
|2017| 8970|
|2018|10188|
|2019| 8492|
|2020| 9345|
|2021| 7544|
+----+-----+

 = 강원도 강릉시 =
+----+-----+
|   y|price|
+----+-----+
|2010| 9305|
|2011|10595|
|2012|11266|
|2013|10635|
|2014|10613|
|2015|11794|
|2016|12604|
|2017|13445|
|2018|14193|
|2019|14259|
|2020|15631|
|2021|17428|
+----+-----+

 = 강원도 평창군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 6311|
|2011| 5043|
|2012| 6373|
|2013| 9

+----+-----+
|   y|price|
+----+-----+
|2010|28534|
|2011|27904|
|2012|24415|
|2013|24770|
|2014|25609|
|2015|27709|
|2016|30118|
|2017|32655|
|2018|36988|
|2019|40973|
|2020|44671|
|2021|49275|
+----+-----+

 = 경기도 오산시 =
+----+-----+
|   y|price|
+----+-----+
|2010|16055|
|2011|18777|
|2012|19038|
|2013|19462|
|2014|20151|
|2015|20276|
|2016|20952|
|2017|21123|
|2018|20867|
|2019|19968|
|2020|23555|
|2021|26547|
+----+-----+

 = 경기도 안성시 =
+----+-----+
|   y|price|
+----+-----+
|2010|10523|
|2011|10864|
|2012|13310|
|2013|14430|
|2014|13244|
|2015|13647|
|2016|13202|
|2017|13183|
|2018|13537|
|2019|12504|
|2020|13370|
|2021|15397|
+----+-----+

 = 경기도 과천시 =
+----+------+
|   y| price|
+----+------+
|2010| 70174|
|2011| 64120|
|2012| 57200|
|2013| 59022|
|2014| 61058|
|2015| 64887|
|2016| 72570|
|2017| 80365|
|2018|103771|
|2019|122124|
|2020|140844|
|2021|162946|
+----+------+

 = 경기도 시흥시 =
+----+-----+
|   y|price|
+----+-----+
|2010|17075|
|2011|17254|
|2012|17051|
|2013|17270|
|2014

+----+-----+
|   y|price|
+----+-----+
|2010|16199|
|2011|19285|
|2012|18612|
|2013|17824|
|2014|18202|
|2015|18615|
|2016|16983|
|2017|18164|
|2018|17848|
|2019|18431|
|2020|19517|
|2021|20833|
+----+-----+

 = 경상남도 남해군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 6547|
|2011| 8713|
|2012|10227|
|2013| 8968|
|2014| 7876|
|2015|10568|
|2016|11064|
|2017|11233|
|2018|10167|
|2019|15832|
|2020|15995|
|2021|10164|
+----+-----+

 = 경상남도 양산시 =
+----+-----+
|   y|price|
+----+-----+
|2010|10621|
|2011|12209|
|2012|11869|
|2013|12933|
|2014|14858|
|2015|14702|
|2016|15425|
|2017|16848|
|2018|17518|
|2019|18539|
|2020|21169|
|2021|19404|
+----+-----+

 = 경상남도 함안군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 9248|
|2011|11413|
|2012|12188|
|2013|13124|
|2014|14172|
|2015|10818|
|2016|14069|
|2017|10697|
|2018|12432|
|2019|12261|
|2020|12308|
|2021|12733|
+----+-----+

 = 경상남도 합천군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 5860|
|2011| 7564|
|2012| 9964|
|2013|10979|
|2014|12381|
|201

+----+-----+
|   y|price|
+----+-----+
|2010|22496|
|2011|27807|
|2012|29351|
|2013|27845|
|2014|32345|
|2015|38388|
|2016|38576|
|2017|42670|
|2018|46683|
|2019|44543|
|2020|50515|
|2021|45550|
+----+-----+

 = 대구광역시 서구 =
+----+-----+
|   y|price|
+----+-----+
|2010|13462|
|2011|14691|
|2012|19072|
|2013|19968|
|2014|19677|
|2015|22043|
|2016|21742|
|2017|23014|
|2018|22157|
|2019|23250|
|2020|27337|
|2021|27201|
+----+-----+

 = 대구광역시 달성군 =
+----+-----+
|   y|price|
+----+-----+
|2010|10691|
|2011|12952|
|2012|14181|
|2013|17765|
|2014|18134|
|2015|19103|
|2016|17516|
|2017|20170|
|2018|20352|
|2019|22642|
|2020|23203|
|2021|21538|
+----+-----+

 = 대구광역시 남구 =
+----+-----+
|   y|price|
+----+-----+
|2010|14197|
|2011|16392|
|2012|18926|
|2013|18290|
|2014|20178|
|2015|23633|
|2016|25471|
|2017|25942|
|2018|25708|
|2019|26883|
|2020|31651|
|2021|32864|
+----+-----+

 = 대구광역시 북구 =
+----+-----+
|   y|price|
+----+-----+
|2010|12168|
|2011|15083|
|2012|16281|
|2013|17870|
|2014|19474|
|20

+----+------+
|   y| price|
+----+------+
|2010| 80262|
|2011| 74935|
|2012| 70802|
|2013| 66239|
|2014| 77198|
|2015| 77456|
|2016| 97189|
|2017|108376|
|2018|132285|
|2019|151887|
|2020|152196|
|2021|184147|
+----+------+

 = 서울특별시 구로구 =
+----+-----+
|   y|price|
+----+-----+
|2010|31467|
|2011|32042|
|2012|30306|
|2013|31092|
|2014|32183|
|2015|33595|
|2016|35771|
|2017|39120|
|2018|43347|
|2019|51021|
|2020|54445|
|2021|60346|
+----+-----+

 = 서울특별시 서대문구 =
+----+-----+
|   y|price|
+----+-----+
|2010|35832|
|2011|34843|
|2012|34654|
|2013|34208|
|2014|35737|
|2015|38146|
|2016|44893|
|2017|49921|
|2018|56375|
|2019|68039|
|2020|78492|
|2021|88164|
+----+-----+

 = 서울특별시 성북구 =
+----+-----+
|   y|price|
+----+-----+
|2010|34757|
|2011|36689|
|2012|34143|
|2013|34371|
|2014|36196|
|2015|39245|
|2016|42893|
|2017|44414|
|2018|49752|
|2019|58509|
|2020|68711|
|2021|81208|
+----+-----+

 = 서울특별시 은평구 =
+----+-----+
|   y|price|
+----+-----+
|2010|41635|
|2011|36872|
|2012|34581|
|2013|340

+----+-----+
|   y|price|
+----+-----+
|2010|14705|
|2011|15392|
|2012|14636|
|2013|15776|
|2014|16241|
|2015|17426|
|2016|17911|
|2017|18196|
|2018|18309|
|2019|17864|
|2020|18283|
|2021|19228|
+----+-----+

 = 인천광역시 부평구 =
+----+-----+
|   y|price|
+----+-----+
|2010|23477|
|2011|23413|
|2012|22259|
|2013|20327|
|2014|21073|
|2015|22328|
|2016|23505|
|2017|24633|
|2018|25844|
|2019|26881|
|2020|28351|
|2021|30640|
+----+-----+

   == 전라남도 ==
 = 전라남도 영광군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 4144|
|2011| 6948|
|2012| 7629|
|2013| 9804|
|2014| 8387|
|2015| 9062|
|2016|10105|
|2017| 8808|
|2018|11587|
|2019|11006|
|2020|11162|
|2021|12555|
+----+-----+

 = 전라남도 진도군 =
+----+-----+
|   y|price|
+----+-----+
|2010| 7172|
|2011| 8368|
|2012| 9260|
|2013|10061|
|2014| 8692|
|2015| 8701|
|2016| 8646|
|2017|10294|
|2018|10090|
|2019|10878|
|2020|12297|
|2021|13865|
+----+-----+

 = 전라남도 나주시 =
+----+-----+
|   y|price|
+----+-----+
|2010| 8148|
|2011| 7601|
|2012| 8354|
|2013| 8549|
|2

+----+-----+
|   y|price|
+----+-----+
|2010| 7513|
|2011|10750|
|2012| 9739|
|2013|13596|
|2014|13266|
|2015|17570|
|2016|20334|
|2017|21398|
|2018|20501|
|2019|22992|
|2020|24199|
|2021|25493|
+----+-----+

 = 제주특별자치도 제주시 =
+----+-----+
|   y|price|
+----+-----+
|2010|13779|
|2011|15175|
|2012|14935|
|2013|15943|
|2014|17339|
|2015|21153|
|2016|26424|
|2017|28962|
|2018|28661|
|2019|28122|
|2020|30871|
|2021|30004|
+----+-----+

   == 충청남도 ==
 = 충청남도 예산군 =
+----+-----+
|   y|price|
+----+-----+
|2010|11985|
|2011|11743|
|2012| 9632|
|2013| 9986|
|2014|10435|
|2015|11304|
|2016|10063|
|2017|10661|
|2018|10872|
|2019|11813|
|2020|13377|
|2021|12712|
+----+-----+

 = 충청남도 천안서북구 =
+----+-----+
|   y|price|
+----+-----+
|2010|15625|
|2011|15483|
|2012|16151|
|2013|17217|
|2014|18297|
|2015|18122|
|2016|16967|
|2017|17660|
|2018|19410|
|2019|23250|
|2020|26713|
|2021|21491|
+----+-----+

 = 충청남도 당진시 =
+----+-----+
|   y|price|
+----+-----+
|2010|16799|
|2011|18580|
|2012|17727|
|2013|17428

# 선형 모델링

In [16]:
feature_columns = df1.columns[:-1]

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="years")

In [17]:
df1.printSchema()

root
 |-- year: string (nullable = true)
 |-- price: double (nullable = true)



- column type int로 변경

In [18]:
from pyspark.sql.types import DateType

In [19]:
df1 = df1.withColumn("year", df1['year'].cast('int'))
df1.show()
print(df1.schema)

+----+-----------------+
|year|            price|
+----+-----------------+
|2010|840.1818181818181|
|2011|814.2857142857143|
|2012|773.3333333333334|
|2013|403.9130434782609|
|2014|855.4444444444445|
|2015|640.5882352941177|
|2016|822.1428571428571|
|2017|868.5714285714286|
|2018|            950.0|
|2019|            887.5|
|2020|            900.0|
|2021|            800.0|
+----+-----------------+

StructType(List(StructField(year,IntegerType,true),StructField(price,DoubleType,true)))


In [20]:
data = assembler.transform(df1)

In [21]:
data.show()

+----+-----------------+--------+
|year|            price|   years|
+----+-----------------+--------+
|2010|840.1818181818181|[2010.0]|
|2011|814.2857142857143|[2011.0]|
|2012|773.3333333333334|[2012.0]|
|2013|403.9130434782609|[2013.0]|
|2014|855.4444444444445|[2014.0]|
|2015|640.5882352941177|[2015.0]|
|2016|822.1428571428571|[2016.0]|
|2017|868.5714285714286|[2017.0]|
|2018|            950.0|[2018.0]|
|2019|            887.5|[2019.0]|
|2020|            900.0|[2020.0]|
|2021|            800.0|[2021.0]|
+----+-----------------+--------+



In [22]:
train, test = data.randomSplit([1.0, 0.0])

In [23]:
train.show()

+----+-----------------+--------+
|year|            price|   years|
+----+-----------------+--------+
|2010|840.1818181818181|[2010.0]|
|2011|814.2857142857143|[2011.0]|
|2012|773.3333333333334|[2012.0]|
|2013|403.9130434782609|[2013.0]|
|2014|855.4444444444445|[2014.0]|
|2015|640.5882352941177|[2015.0]|
|2016|822.1428571428571|[2016.0]|
|2017|868.5714285714286|[2017.0]|
|2018|            950.0|[2018.0]|
|2019|            887.5|[2019.0]|
|2020|            900.0|[2020.0]|
|2021|            800.0|[2021.0]|
+----+-----------------+--------+



In [24]:
test.show()

+----+-----+-----+
|year|price|years|
+----+-----+-----+
+----+-----+-----+



In [25]:
from pyspark.ml.regression import LinearRegression

In [26]:
algo = LinearRegression(featuresCol="years", labelCol="price")

In [27]:
model = algo.fit(train)

In [28]:
#predictions = model.transform(test)
#predictions.select(predictions.columns[-2:]).show()

# 예측 set 생성

In [29]:
from pyspark.sql import Row
# from pyspark.mllib.linalg import DenseVector

In [30]:
test1 = [Row(year=2022), Row(year=2023)]

In [31]:
test1 = spark.createDataFrame(test1)

In [32]:
test1 = assembler.transform(test1)

In [33]:
test1.show()

+----+--------+
|year|   years|
+----+--------+
|2022|[2022.0]|
|2023|[2023.0]|
+----+--------+



In [34]:
predictions = model.transform(test1)

In [35]:
predictions.select(predictions.columns[:]).show()

+----+--------+-----------------+
|year|   years|       prediction|
+----+--------+-----------------+
|2022|[2022.0]| 889.056563687871|
|2023|[2023.0]|903.3221776561077|
+----+--------+-----------------+

